In [9]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"
saved_mab_model = "checkpoints/mab_model_200.pth"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [52]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation."
user_input = "I am extremely disappointed with the quality; it broke after just one day."

content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

I am extremely disappointed with the quality; it broke after just one day.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

negative<|eot_id|>


In [53]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm, hidden_size=1024)
mab_model.to(device)
print()

/tmp/1104405.1.gpu/ipykernel_2110940/3922162976.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), ll

In [54]:
gen_inputs = tokenizer(gen_output, return_tensors="pt").to(device)
input_ids = gen_inputs['input_ids'][:, :-1]
attention_mask = gen_inputs['attention_mask'][:, :-1]
_, mab_values, _ = mab_model.get_dist_value(input_ids, attention_mask)



In [55]:
print(mab_values)
print(mab_values.shape)

tensor([[-0.0048, -0.0048, -0.0048, -0.0195, -0.0108, -0.0115, -0.0037, -0.0281,
         -0.0132, -0.0254,  0.0010, -0.0036, -0.0132, -0.0204, -0.0094, -0.0220,
         -0.0097, -0.0097, -0.0091,  0.0034, -0.0053, -0.0037, -0.0249, -0.0055,
         -0.0102, -0.0329, -0.0148, -0.0046, -0.0128, -0.0081, -0.0189, -0.0085,
         -0.0114, -0.0161, -0.0225, -0.0132, -0.0071, -0.0023, -0.0066, -0.0093,
         -0.0100, -0.0085, -0.0133, -0.0178,  0.0026, -0.0078, -0.0145, -0.0023,
         -0.0010,  0.0029, -0.0092, -0.0080, -0.0131, -0.0132, -0.0084, -0.0125,
         -0.0145, -0.0141, -0.0102, -0.0163, -0.0077, -0.0102, -0.0158, -0.0158,
         -0.0085, -0.0127, -0.0113, -0.0081, -0.0071, -0.0152, -0.0093, -0.0006,
         -0.0461, -0.0133, -0.0203, -0.0335,  0.0086,  0.0023, -0.0046, -0.0136,
         -0.0241, -0.0340, -0.0138,  0.0002, -0.0109, -0.0107, -0.0188, -0.0308,
         -0.0182, -0.0201, -0.0178, -0.0178,  0.0136, -0.0051, -0.0090]],
       device='cuda:0', grad_fn=<Sq

In [56]:
print(input_ids)
print(input_ids.shape)

tensor([[128000, 128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,
          33025,   2696,     25,   6790,    220,   2366,     18,    198,  15724,
           2696,     25,    220,   1627,  10263,    220,   2366,     19,    271,
           2127,  56956,    279,  27065,    315,    279,   2768,  11914,    323,
           6013,    449,   1193,    832,   3492,     25,    364,  31587,   2965,
            364,  43324,   2965,    477,    364,  60668,   2965,   3196,    389,
            279,   8244,  16630,    323,   7438,    315,    279,  11914,     13,
           3234,    539,   3493,    904,   5217,  16540,     13, 128009, 128006,
          52989, 128007,    271,     40,   1097,   9193,  25406,    449,    279,
           4367,     26,    433,  14760,   1306,   1120,    832,   1938,     13,
         128009, 128006,  78191, 128007,    271,  43324]], device='cuda:0')
torch.Size([1, 96])


In [57]:
def visualize_tokens_with_values(input_ids, mab_values, tokenizer):
    # Decode tokens one by one to preserve alignment
    tokens = []
    for i in range(input_ids.shape[1]):
        token = tokenizer.decode(input_ids[0, i:i+1])
        tokens.append(token)
    
    # Normalize MAB values to [0,1] for color intensity first
    normalized_values = (mab_values[0] - mab_values[0].min()) / (mab_values[0].max() - mab_values[0].min())
    
    # Pad normalized_values with a zero at the end
    padded_normalized_values = torch.cat([normalized_values, torch.zeros(1, device=mab_values.device)], dim=0)
    # Pad original mab_values with the last actual value
    padded_mab_values = torch.cat([mab_values[0], mab_values[0][-1:]], dim=0)
    
    # Generate HTML with colored text and values
    html_output = "<div style='font-family: monospace; line-height: 2; background-color: white; padding: 10px;'>"
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        # Use a gradient from white to green
        intensity = float(value)
        green_color = int(intensity * 200)  # Control the maximum intensity
        html_output += f'<span style="color: black; background-color: rgba(0, {green_color}, 0, 0.3); padding: 0.2em; margin: 0.1em; border-radius: 3px;" title="MAB: {orig_value:.3f}, Norm: {value:.3f}">{token}</span>'
    html_output += "</div>"
    
    # Print the values
    print("Token\tNormalized Value\tOriginal MAB Value")
    print("-" * 50)
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        print(f"{token}\t{value:.3f}\t\t{orig_value:.3f}")
    
    from IPython.display import HTML
    return HTML(html_output)

In [58]:
# Usage:
visualization = visualize_tokens_with_values(input_ids, mab_values, tokenizer)
display(visualization)

Token	Normalized Value	Original MAB Value
--------------------------------------------------
<|begin_of_text|>	0.691		-0.005
<|begin_of_text|>	0.691		-0.005
<|begin_of_text|>	0.691		-0.005
<|start_header_id|>	0.445		-0.020
system	0.591		-0.011
<|end_header_id|>	0.579		-0.012


	0.710		-0.004
Cut	0.302		-0.028
ting	0.551		-0.013
 Knowledge	0.346		-0.025
 Date	0.789		0.001
:	0.712		-0.004
 December	0.551		-0.013
 	0.430		-0.020
202	0.615		-0.009
3	0.404		-0.022

	0.609		-0.010
Today	0.610		-0.010
 Date	0.619		-0.009
:	0.829		0.003
 	0.683		-0.005
26	0.710		-0.004
 Jul	0.355		-0.025
 	0.680		-0.005
202	0.601		-0.010
4	0.222		-0.033


	0.525		-0.015
An	0.695		-0.005
alyze	0.557		-0.013
 the	0.636		-0.008
 sentiment	0.455		-0.019
 of	0.629		-0.009
 the	0.582		-0.011
 following	0.502		-0.016
 sentence	0.396		-0.022
 and	0.551		-0.013
 respond	0.653		-0.007
 with	0.733		-0.002
 only	0.662		-0.007
 one	0.617		-0.009
 word	0.605		-0.010
:	0.630		-0.008
 '	0.550		-0.013
positive	0.473		-0.018
,'